In [ ]:
# !wget https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt
# !wget https://www.dropbox.com/s/ripsc7hfc0nihy5/test.txt

In [ ]:
import random
 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LSTM, GRU, CuDNNGRU, CuDNNLSTM 
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

from keras.losses import categorical_crossentropy
from keras.utils import to_categorical

import tensorflow as tf
import os

In [ ]:
# TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [ ]:
path = './ferdosi.txt'
with open(path, encoding='utf-8') as f:
    texts = f.read()

beits = texts.split('\n')
# texts = texts.split('\n')
texts = texts.replace('\n', ' ــ ')
texts = texts.split(' ')
# texts = text_to_word_sequence(texts)

print('corpus length:', len(texts))

In [ ]:
texts[:10]

In [ ]:
max_length = 6
embeding_dim = 32
step = 1


tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences = [sequences[i][0] for i in range(len(sequences)-1)]

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

len_of_words = len(word_index)
word2idx = word_index
idx2word = dict([(value, key) for (key, value) in word_index.items()])

# paded_sequences = pad_sequences(sequences, maxlen=max_length)
# paded_sequences = [paded_sequences[i][0] for i in range(len(paded_sequences)-1)]

In [ ]:
sequences[:10]

In [ ]:

sentences = []
next_word = []
for i in range(0, len(sequences) - max_length, step):
    sentences.append(sequences[i: i + max_length])
    next_word.append(sequences[i + max_length])
print('nb sequences:', len(sequences))

In [ ]:
for ii in range(3):

    print(sentences[ii])
    print([idx2word.get(i, '') for i in sentences[ii]])

    print(next_word[ii])
    print(idx2word[next_word[ii]])


In [ ]:
x = np.array(sentences)
y = np.array(next_word)
# y = to_categorical(y)

In [ ]:
def on_epoch_end(epoch, _):
#     if epoch % 10 != 0:
#         return
    
    start_index = random.randint(0, len(beits) - 1)
    for diversity in [1.0]:
        print('----- diversity:', diversity)

        sentence = beits[start_index]
        print(sentence)
        generated = sentence
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(max_length * 10):
            x_pred = np.zeros((1, max_length), dtype=int)
            for t, word in enumerate(sentence.split(' ')):
                try:
#                     print(t, word)
                    x_pred[0, t] = word2idx[word]
                except:
                    pass

            preds = model.predict(x_pred, verbose=0)[0]
            pred_id = sample(preds, diversity)
            next_word = idx2word[pred_id]

            generated = generated + ' ' + next_word
            sentence = ' '.join(sentence.split(' ')[1:])
            sentence = sentence + ' ' + next_word

        print(generated.replace('ــ', '\n'))

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# model = models.Sequential()
# model.add(layers.Embedding(len(word_index), 100))
# model.add(layers.LSTM(128))
# model.add(layers.Dense(1, activation='sigmoid'))

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


model = Sequential()
model.add(Embedding(len_of_words, embeding_dim))
if tf.test.is_gpu_available():
    # model.add(CuDNNGRU(256, return_sequences=True))
    model.add(CuDNNGRU(256))
else:
    model.add(GRU(256))
model.add(Dense(len_of_words, activation='softmax'))
 
model.compile(optimizer=RMSprop(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
print(model.summary())
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:

# strategy = tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
# model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)


In [ ]:

history = model.fit(x=x, y=y, epochs=60, batch_size=128, callbacks=[print_callback])
